In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
import os
import sys
import time
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from IPython.display import clear_output

class Client:
    def __init__(self):
        self.client = None
        self.wait_driver = None
        
    def create_client(self, options={}):
        if self.client:
            return
        
        options = webdriver.ChromeOptions()
        self.client = webdriver.Chrome(ChromeDriverManager().install(), options=options)
        self.client.implicitly_wait(15)
        self.wait_driver = WebDriverWait(self.client, timeout=10)
        
        return self.client
    
    def close_client(self):
        if not self.client:
            return
        
        self.client.quit()

class Scraper(Client):
    
    def __init__(self):
        super().__init__()
    
    def get_data(self):
        if not self.client:
            cli = self.create_client()
        else:
            cli = self.client
            
        cli.get('https://broadbandmap.fcc.gov/data-download/nationwide-data')
        select = Select(cli.find_element(By.ID, 'state'))
        #52
        for i in range(1, 5):
            select.select_by_index(i)
        
            divs = cli.find_elements(By.CSS_SELECTOR, 'div.border.rounded.p-3.h-100')

            # for d in divs:
            #     txt = d.find_element(By.TAG_NAME, 'h2').text
            #     print(txt)
            #     if txt == "Fixed Broadband Availability Data":
            #         target = d
            #         break
            target = divs[0]
        
            table = target.find_elements(By.CSS_SELECTOR, 'tbody tr')
            
            for t in table:
                if 'Cable' in t.text or 'Fiber' in t.text:
                    t.find_element(By.TAG_NAME, 'button').click()
                    time.sleep(2)

        # table[0].find_element(By.TAG_NAME, 'button').click()
        time.sleep(10)
    
        
        
        

In [64]:
scrape = Scraper()

In [65]:
scrape.create_client()
scrape.get_data()
scrape.close_client()

In [68]:
import json
with open('../misc/fips.txt') as file:
    lines = file.readlines()
    obj = {}
    for line in lines:
        key = line[:2]
        val = line[2:].strip()
        obj[key] = val
    
with open('../misc/fips.json', 'w') as file:
    j = json.dumps(obj, indent=3)
    
    file.write(j)

In [73]:
import os

with open('../misc/fips.json', 'r') as file:
    obj = json.loads(file.read())

# os.rename('../misc/test.txt', '../misc/done.txt')

direct = os.listdir('../FCC-DATA')

for d in direct:
    fip = d[4:6]
    os.rename(f'../FCC-DATA/{d}', f'../FCC-DATA/{obj[fip]}.zip')
